In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Lets check your GPU
if tf.config.list_logical_devices('GPU') != []:
    print("LET'S DO THISS!!! :DDDD")
else: 
    print("Oh dude... D:")

LET'S DO THISS!!! :DDDD


In [33]:
with open('data/input.txt') as f:
    content = f.read().lower()
    
print(content[:100])

first citizen:
before we proceed any further, hear me speak.

all:
speak, speak.

first citizen:
you


In [34]:
"".join(sorted(set(content.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [35]:
text_vec_layer = keras.layers.TextVectorization(
                                                split="character",
                                                standardize="lower"
                                            )
text_vec_layer.adapt([content])
encoded = text_vec_layer([content])[0]

In [36]:
encoded -= 2  # drop tokens 0 (pad) and 1 (unknown), subtraindo 2 (pad e unknown)
n_tokens = text_vec_layer.vocabulary_size() - 2  # number of distinct chars = 39
dataset_size = len(encoded)  # total number of chars = 1,115,394

In [37]:
n_tokens, dataset_size

(39, 1115394)

In [38]:
window_size = 128
batch_size = 32
ds = tf.data.Dataset.from_tensor_slices(encoded) # Fatia o ds em janelas de tamanho window_size
ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
ds = ds.shuffle(100_000, seed=42)
ds = ds.batch(batch_size)
ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [39]:
for X_batch, y_batch in ds.take(1):
    print(X_batch.shape, y_batch.shape)
    print(X_batch[0], y_batch[0])

(32, 128) (32, 128)
tf.Tensor(
[15  0  9  3 16 17  0  9  3  0 14  3  8  1 26 10 10 18  3 14  5  9  5 13
  7 23 10 11  3  3 24 17  0  7  5  8 17  0 15  3 13  8  0 14  3  2  6  1
  8 30 10 10 18  3  8  5  3 11  4  9 13  7 23 10  3 17 10 15  3 13  0  6
  4 25  1 17  0  5  0 24  9  3 16 17  0 22  1  2  5  2  5  3  9 27 12  0
  4 11 11  0  2  6  1  0 20  3 12  7 10 19  3  8  0 14 15  0 22  8  3  7
 22  1  8  5  2 15 30 10], shape=(128,), dtype=int64) tf.Tensor(
[ 0  9  3 16 17  0  9  3  0 14  3  8  1 26 10 10 18  3 14  5  9  5 13  7
 23 10 11  3  3 24 17  0  7  5  8 17  0 15  3 13  8  0 14  3  2  6  1  8
 30 10 10 18  3  8  5  3 11  4  9 13  7 23 10  3 17 10 15  3 13  0  6  4
 25  1 17  0  5  0 24  9  3 16 17  0 22  1  2  5  2  5  3  9 27 12  0  4
 11 11  0  2  6  1  0 20  3 12  7 10 19  3  8  0 14 15  0 22  8  3  7 22
  1  8  5  2 15 30 10 10], shape=(128,), dtype=int64)


In [40]:
# Criando função
def to_dataset(sequence, window_size=128, batch_size=32, seed=42, shuffle=False, target='all_window'):
    ds = tf.data.Dataset.from_tensor_slices(sequence) # Fatia o ds em janelas de tamanho window_size
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True) # Você pode visualizar com um .as_numpy_iterator() e colocar num loop com .take(1)
    ds = ds.flat_map(lambda window: window.batch(window_size + 1)) # Transforma cada janela em um tensor
    if shuffle:
        ds = ds.shuffle(100_000, seed=seed)
    ds = ds.batch(batch_size)
    if target == 'all_window':
        ds = ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)
    elif target == 'last_char':
        ds = ds.map(lambda window: (window[:, :-1], window[:, -1])).prefetch(1)
    else:
        raise ValueError('target must be "all_window" or "last_char"')
    return ds

In [41]:
# Exemplo de uso
list(to_dataset(text_vec_layer(['To be'])[0], window_size=3))

[(<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 4,  5,  2],
         [ 5,  2, 23]], dtype=int64)>,
  <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[ 5,  2, 23],
         [ 2, 23,  3]], dtype=int64)>)]

In [42]:
window_size = 128
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], window_size=window_size, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], window_size=window_size)
test_set = to_dataset(encoded[1_060_000:], window_size=window_size)

### Training Char-RNN model

In [74]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.Conv1D(32, kernel_size=3, padding="causal", activation="relu"),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [75]:
def decoder(seq, delta=2):
    output = np.array(text_vec_layer.get_vocabulary())[seq + delta]
    new_output = []
    for i in range(output.shape[0]):
        phrase = ""
        for j in range(output.shape[1]):
            phrase += output[i, j]
        new_output.append(phrase)
    return np.array(new_output)

In [76]:
# estilo
plt.style.use('dark_background')
plt.rcParams['axes.facecolor'] = '#26262e' 

def print_multiple_generated(predicted_text, original_text, n=8):
    print("--> Original text:")
    for i in range(n):
        print(original_text[i])
    print("\n--> Generated text:")
    for i in range(n):
        print(predicted_text[i])
        
def plot_loss(history, step):
    plt.plot(list(range(len(history['loss']))), history['loss'], label='Training Loss')
    plt.plot(list(range(len(history['loss']))), history['val_loss'], label='Validation Loss')
    plt.xlabel('Step')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def plot_accuracy(history, step):
    plt.plot(list(range(len(history['accuracy']))), history['accuracy'], label='Training Accuracy')
    plt.plot(list(range(len(history['accuracy']))), history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Step')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [77]:
history = {
            'text_base': 'To be or not ',
            'text_base_encoded': text_vec_layer(['To be or not ']),
            'list_predicted_text_base': [],
        }

class LogCallback(tf.keras.callbacks.Callback):
    def __init__(self, history):
        super().__init__()
        self.history_ = history
    
    def on_epoch_end(self, epoch, logs=None):
        print(f"Final da época {epoch}, Loss: {logs['loss']}, Acc: {logs['accuracy']}")
    
    def on_train_batch_end(self, batch, logs=None):
        if batch % 1_000 == 0:
            print("")
            print(f"Final do lote {batch}, Loss do lote: {logs['loss']}")
            model = self.model
            # predicted_text = decoder(tf.argmax(model(history['text_base_encoded'], training=False), axis=2))[0]
            predicted_text = extend_text("To be or not ", temperature=1.0)
            print("")
            print("--> Original text: ", history['text_base'])
            print("--> Generated text: ", predicted_text)
            self.history_['list_predicted_text_base'].append(predicted_text)
            print("")

In [78]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True)

model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="nadam",
              metrics=["accuracy"])

shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    model
])

tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU

def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text], verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [79]:
history_train = model.fit(train_set, 
                    validation_data=valid_set, 
                    epochs=10,
                    callbacks=[LogCallback(history=history), 
                               model_ckpt])

Epoch 1/10



Final do lote 0, Loss do lote: 3.661360502243042

--> Original text:  To be or not 
--> Generated text:  To be or not !hq?d
ddidn:;&yoe-3
j.&lvj,s-pxh. v:kxgo? woystj3


      5/Unknown - 26s 44ms/step - loss: 3.6487 - accuracy: 0.1231WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0261s vs `on_train_batch_end` time: 0.8879s). Check your callbacks.


    998/Unknown - 48s 22ms/step - loss: 2.1369 - accuracy: 0.3838
Final do lote 1000, Loss do lote: 2.1355979442596436

--> Original text:  To be or not 
--> Generated text:  To be or not ouch brod heart.

coriolanus:
have oe their rasery

   2000/Unknown - 75s 25ms/step - loss: 1.8502 - accuracy: 0.4560
Final do lote 2000, Loss do lote: 1.8500012159347534

--> Original text:  To be or not 
--> Generated text:  To be or not peacem--
face? i good hart bear med him out.-
cori

   3000/Unknown - 104s 26ms/step - loss: 1.7223 - accuracy: 0.4887
Final do lote 3000, Loss do lote: 1.7222559452056885

--> Original text:  To be or not 
--> Generated text:  To be or not all that men thee? say, say her to reven end;
i' t

   4000/Unknown - 131s 26ms/step - loss: 1.6455 - accuracy: 0.5088
Final do lote 4000, Loss do lote: 1.6454747915267944

--> Original text:  To be or not 
--> Generated text:  To be or not tasen's lost are giving as so-pebspities to pard s

   4999/Unknown - 158s 26ms/step - los

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31246/31246 [==============================] - 841s 26ms/step - loss: 1.2367 - accuracy: 0.6115 - val_loss: 1.5484 - val_accuracy: 0.5510
Epoch 7/10

Final do lote 0, Loss do lote: 1.5951448678970337

--> Original text:  To be or not 
--> Generated text:  To be or not biund,
that had cusely mask thy inself.
sirlay him

 1000/31246 [..............................] - ETA: 11:04 - loss: 1.2576 - accuracy: 0.6061
Final do lote 1000, Loss do lote: 1.2575058937072754

--> Original text:  To be or not 
--> Generated text:  To be or not so, so 't so, mad;
think him need weal'd in here p

 1998/31246 [>.............................] - ETA: 11:47 - loss: 1.2304 - accuracy: 0.6126
Final do lote 2000, Loss do lote: 1.230389952659607

--> Original text:  To be or not 
--> Generated text:  To be or not up his own
tribunes.

lartius:
then they of the va

 2999/31246 [=>............................] - ETA: 11:40 - loss: 1.2238 - accuracy: 0.6143
Final do lote 3000, Loss do lote: 1.2237807512283325

--

INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


31246/31246 [==============================] - 829s 26ms/step - loss: 1.2228 - accuracy: 0.6146 - val_loss: 1.5482 - val_accuracy: 0.5511
